In [3]:
import numpy as np
import pandas as pd

# Función de activación escalón
def escalon(x):
    return np.where(x >= 0, 1, 0)

# Derivada de la función escalón (Para fines de backpropagation, usamos una aproximación)
def escalon_derivada(x):
    return np.ones_like(x)  # Esto es una aproximación, ya que la derivada real del escalón no es útil en backprop

# Lectura del conjunto de datos Iris
datos_iris = pd.read_csv('iris.csv')

# Normalización de los datos
datos = datos_iris.iloc[:, :-1]
datos = (datos - datos.mean()) / datos.std()

# Agregamos una columna de unos para el sesgo
datos['sesgo'] = 1

# Convertimos los datos a matriz numpy
X = datos.values

# Etiquetas (target)
y = datos_iris['target'].values

# Codificación one-hot de las etiquetas
y_encoded = np.zeros((y.size, y.max() + 1))
y_encoded[np.arange(y.size), y] = 1

# Definición de parámetros
tamano_entrada = X.shape[1]  # número de columnas de X
tamano_salida = 3  # 3 clases en el conjunto de datos Iris
tamano_oculta1 = 6  # capa oculta 1
tamano_oculta2 = 5  # capa oculta 2
tamano_oculta3 = 4  # capa oculta 3
tasa_aprendizaje = 0.2
epocas = 4

# Inicialización de pesos
np.random.seed(37)
pesos_entrada_oculta1 = np.random.rand(tamano_entrada, tamano_oculta1) - 0.5
pesos_oculta1_oculta2 = np.random.rand(tamano_oculta1, tamano_oculta2) - 0.5
pesos_oculta2_oculta3 = np.random.rand(tamano_oculta2, tamano_oculta3) - 0.5
pesos_oculta3_salida = np.random.rand(tamano_oculta3, tamano_salida) - 0.5

# Entrenamiento de la red neuronal
for epoca in range(epocas):
    # Forward pass (producto punto y activación)
    entrada_oculta1 = np.dot(X, pesos_entrada_oculta1)
    salida_oculta1 = escalon(entrada_oculta1)

    entrada_oculta2 = np.dot(salida_oculta1, pesos_oculta1_oculta2)
    salida_oculta2 = escalon(entrada_oculta2)

    entrada_oculta3 = np.dot(salida_oculta2, pesos_oculta2_oculta3)
    salida_oculta3 = escalon(entrada_oculta3)

    entrada_salida = np.dot(salida_oculta3, pesos_oculta3_salida)
    salida_final = escalon(entrada_salida)

    # Calcular el error
    error = y_encoded - salida_final

    # Backpropagation (errores y actualización de pesos)
    delta_salida = error * escalon_derivada(salida_final)
    error_oculta3 = delta_salida.dot(pesos_oculta3_salida.T)
    delta_oculta3 = error_oculta3 * escalon_derivada(salida_oculta3)
    error_oculta2 = delta_oculta3.dot(pesos_oculta2_oculta3.T)
    delta_oculta2 = error_oculta2 * escalon_derivada(salida_oculta2)
    error_oculta1 = delta_oculta2.dot(pesos_oculta1_oculta2.T)
    delta_oculta1 = error_oculta1 * escalon_derivada(salida_oculta1)

    # Actualizar pesos
    pesos_oculta3_salida += salida_oculta3.T.dot(delta_salida) * tasa_aprendizaje
    pesos_oculta2_oculta3 += salida_oculta2.T.dot(delta_oculta3) * tasa_aprendizaje
    pesos_oculta1_oculta2 += salida_oculta1.T.dot(delta_oculta2) * tasa_aprendizaje
    pesos_entrada_oculta1 += X.T.dot(delta_oculta1) * tasa_aprendizaje

    # Imprimir la pérdida
    if (epoca + 1) % 10 == 0:
        perdida = np.mean(np.abs(error))
        print(f'Época {epoca + 1}, Pérdida: {perdida}')

# Predicciones
predicciones = np.argmax(salida_final, axis=1)

# Imprimir las predicciones
resultados = pd.DataFrame({
    'Etiqueta Real': y,
    'Predicción': predicciones
})

print(resultados)


     Etiqueta Real  Predicción
0                0           0
1                0           0
2                0           0
3                0           0
4                0           0
..             ...         ...
145              2           1
146              2           1
147              2           1
148              2           1
149              2           1

[150 rows x 2 columns]
